<h1>User-User</h1>

<h1>Khuyến nghị 10 bộ phim cho UserID = 20</h1>

In [2]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
from math import sqrt

In [3]:
movies = pd.read_csv(r"data\movie\movies.csv").head(1000)
ratings = pd.read_csv(r"data\movie\ratings.csv").head(3000)

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
def get_rating(userId, movieId):
    # Lấy rating của user cho 1 bộ phim
    try:
        return ratings.loc[(ratings.userId == userId) & (ratings.movieId == movieId), 'rating'].iloc[0]
    except:
        return 0

In [7]:
get_rating(1, 1)

4.0

In [8]:
def consine(a: list, b: list):
    if a == [] or b == []:
        return 0
    # Tính consine của 2 vectors
    if a == b:
        return 1
    if norm(a)*norm(b) == 0:
        return 0
    return np.dot(a,b)/(norm(a)*norm(b))


In [9]:
def pearson(x: list, y: list):
    # Tính pearson của 2 vectors
    if x == [] or y == []:
        return 0
    if x == y:
        return 1
    avg_x = sum(x)/len(x)
    avg_y = sum(y)/len(y)

    err_x = [i - avg_x for i in x]
    err_y = [i - avg_y for i in y]

    a = sum([i*j for i, j in zip(err_x, err_y)])
    b1 = sum([i*i for i in err_x])
    b2 = sum([j*j for j in err_y])

    if (sqrt(b1)*sqrt(b2)) == 0:
        return 0

    return a/(sqrt(b1)*sqrt(b2))

In [10]:
def mutual_movies(userId1, userId2):
    # Tìm danh sách phim xem chung của 2 user
    list_1 = list(ratings.loc[ratings.userId == userId1, 'movieId'])
    list_2 = list(ratings.loc[ratings.userId == userId2, 'movieId'])
    return [i for i in list_1 if i in list_2]

In [11]:
def  userId_watched(movieId):
    # Tìm danh sách các user đã xem 1 bộ phim
    return ratings.loc[ratings.movieId == movieId, 'userId'].to_list()

In [12]:
def similaraty_score(userId1, userId2, measure: str):
    # Tính độ tương tự của 2 user theo consine hoặc pearson
    mutual_list = mutual_movies(userId1, userId2)
    if len(mutual_list) == 0:
        return 0
    vector_1 = [get_rating(userId1, m) for m in mutual_list]
    vector_2 = [get_rating(userId2, m) for m in mutual_list]
    if measure == 'consine':
        return consine(vector_1, vector_2)
    elif measure == 'pearson':
        return pearson(vector_1, vector_2)

In [13]:
def similar_user_list(userId: int, userId_list: list, measure: str):
    # Danh sách độ tương tự của userId với các user khác
    similaraty_score_list = sorted([(i ,similaraty_score(userId, i, measure)) for i in userId_list if i != 1],key = lambda x: x[1], reverse=True)
    return similaraty_score_list

In [14]:
def get_recomendation(userId, movieId, k, measure: str):
    sim_user_list = similar_user_list(userId, userId_watched(movieId), measure)[0:k]
    # Tính rating theo phương pháp đánh giá có trọng số
    tu = sum([ sim*get_rating(userId, movieId) for userId, sim in sim_user_list])
    if tu == 0:
        return 0
    mau = sum([sim for userId, sim in sim_user_list])
    return tu/mau

In [15]:
def get_not_watched_list(userId):
    return set(movies['movieId']).difference(set(ratings.loc[ratings.userId == userId, 'movieId']))

In [16]:
def get_top_n_ranked_movies_list(userId, n, k, measure):
    movies_list = get_not_watched_list(userId)
    recommendations_list = sorted([ ( m ,get_recomendation(userId, m, k, measure)) for m in movies_list], key=lambda x : x[1], reverse=True)
    return recommendations_list[0:10]

In [17]:
ranked_list = get_top_n_ranked_movies_list(20 ,10, 10,'consine')
ranked_list

[(5, 5.000000000000001),
 (22, 5.000000000000001),
 (86, 5.000000000000001),
 (160, 5.000000000000001),
 (168, 5.000000000000001),
 (230, 5.000000000000001),
 (248, 5.000000000000001),
 (257, 5.000000000000001),
 (277, 5.000000000000001),
 (314, 5.000000000000001)]

In [18]:
print('Top 10 bộ phim khuyến nghị cho User20:')
print('--------------------------------------')
for id, predict in ranked_list:
    print(id, movies.loc[movies.movieId == id, 'title'].iloc[0])

Top 10 bộ phim khuyến nghị cho User20:
--------------------------------------
5 Father of the Bride Part II (1995)
22 Copycat (1995)
86 White Squall (1996)
160 Congo (1995)
168 First Knight (1995)
230 Dolores Claiborne (1995)
248 Houseguest (1994)
257 Just Cause (1995)
277 Miracle on 34th Street (1994)
314 Secret of Roan Inish, The (1994)


<h1>Item-Item</h1>

<h1>Top 10 User sẽ đánh cao nhất cho MovieID = 50</h1>

In [30]:

movies = pd.read_csv(r"D:\Documents\Courses\HeKhuyenNghi\PracticalAssignment\BTTH1_19521586\movie\movies.csv")
ratings = pd.read_csv(r"D:\Documents\Courses\HeKhuyenNghi\PracticalAssignment\BTTH1_19521586\movie\ratings.csv").head(50000)

In [20]:
def mutual_users(movieId1, movieId2):
    # Tìm danh users cùng đánh giá 2 bộ phim
    list_1 = list(ratings.loc[ratings.movieId == movieId1, 'userId'])
    list_2 = list(ratings.loc[ratings.movieId == movieId2, 'userId'])
    return [i for i in list_1 if i in list_2]

In [21]:
def similaraty_score_of_2_movies(movieId1, movieId2, measure: str):
    mutual_users_list = mutual_users(movieId1, movieId2)
    vector_1 = [get_rating(i, movieId1) for i in mutual_users_list]
    vector_2 = [get_rating(i, movieId2) for i in mutual_users_list]
    if measure == 'consine':
        return consine(vector_1, vector_2)
    elif measure == 'pearson':
        return pearson(vector_1, vector_2)

In [22]:
def get_most_k_similar_movies_list(movieId: int, movieId_list: list, k: int, measure):
    l = sorted([(i, similaraty_score_of_2_movies(i, movieId, measure)) for i in movieId_list if i != movieId], key=lambda x: x[1], reverse=True)
    return l[0:k]

In [23]:
def  movieId_watched(userId):
    # Tìm danh sách các movies mà user đã xem
    return ratings.loc[ratings.userId == userId, 'movieId'].to_list()

In [24]:
def get_recomendation(movieId, userId, k, measure: str):
    sim_movie_list = get_most_k_similar_movies_list(movieId, movieId_watched(userId), k, 'pearson')
    #  Tính rating theo phương pháp đánh giá có trọng số
    tu = sum([ sim*get_rating(userId, movie) for movie, sim in sim_movie_list])
    if tu == 0:
        return 0
    mau = sum([sim for movieId, sim in sim_movie_list])
    return tu/mau


In [25]:
def get_not_rated_list(movieId):
    # Tìm các users chưa đánh giá cho bộ phim
    all_users = set(ratings['userId'])
    rated_user = set(ratings.loc[ratings.movieId == movieId, 'userId'])
    return all_users.difference(rated_user)

In [26]:
def get_top_n_user_ranked_list(movieId, n, k, measure: str):
    users_list = get_not_rated_list(movieId)
    return sorted([(u, get_recomendation(movieId, u, k, measure)) for u in users_list], key=lambda x: x[1], reverse=True)[0:k]

In [32]:
ranked = get_top_n_user_ranked_list(50, 10, 10, 'pearson')
ranked

[(53, 5.000000000000001),
 (251, 5.000000000000001),
 (171, 4.899611694737756),
 (25, 4.7719460645650855),
 (43, 4.61138875586321),
 (30, 4.592243941235577),
 (236, 4.572988660869799),
 (250, 4.5603809583556805),
 (319, 4.546743844902207),
 (128, 4.516837398462648)]

In [34]:
print('Các UserID được dự đoán sẽ đánh giá cao cho MovieID = 50:')
for i,j in ranked:
    print(i)

Các UserID được dự đoán sẽ đánh giá cao cho MovieID = 50:
53
251
171
25
43
30
236
250
319
128
